<a href="https://colab.research.google.com/github/hansupark/2020_2-AI/blob/master/MCTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#몬테 카를로 트리 탐색 실습 

In [ ]:
from abc import ABC, abstractmethod
from collections import defaultdict
import math

class MCTS:

  def __init__(self, c = 1):
    self.WC = defaultdict(int) # 노드별 이긴 횟수
    self.VC = defaultdict(int) # 노드별 방문 횟수
    self.children = dict() # 노드 자식노드들
    self.c = c #UCB 계산시 사용되는 계수

  def choose(self, node):
    #최선의 자식노드를 선택하는 메소드

    if node.is_terminal(): #해당 노드가 단말 노드일 경우
      raise RuntimeError(f"choose called on ternimal node {node}")
    
    if node not in self.children:  # 해당 노드가 자식 노드들에 포함되지 않을시 
      return node.find_random_chlld()  #무작위로 자식노드를 리턴

    def score(n):
      #점수를 계산하는 메소드

      if self.VC[n] == 0 : #해당 노드의 방문횟수가 0일 경우
        return float("-inf") # 선택 배제
      return self.WC[n] / self.VC[n] # 0회 이상일 경우 평균 값 리턴
    
    return max(self.children[node], key = score) 
    #score 함수 사용시 해당 노드의 자식들 중 최대값을 리턴
    
  def do_rollout(self, node):
    
    path = self._select(node)
    leaf = path[-1]
    self._expand(leaf)
    reward = self._simulate(leaf)
    self._backpropagate(path, reward)

  def _select(self, node):
    #아직 시도 하지 않은 자식 노드 찾음

    path = []
    while True:
      path.append(node)
      if node not in self.children or not self.children(node):
        
        return path
      unexplored = self.children[node] - self.children.keys()

      if unexplored :
        n = unexplored.pop()
        path.append(n)

        return path
      
      node = self._uct_select(node)